In [1]:
import polars as pl
import altair as alt

In [3]:
import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [5]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [7]:
def vyvoj_vyskytu(hledani, popisek):
    
    return df.with_columns(
        pl.col("text").str.count_matches(hledani).alias("vyskytu")
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("vyskytu").sum()
    ).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("vyskytu") / pl.col("pocet_slov")).alias("pomer")
).with_columns(
    pl.lit(popisek).alias("co")
)

In [605]:
def porovnani_vyskytu(regex_1,popis_1,regex_2,popis_2):
    return vyvoj_vyskytu(regex_1,popis_1).join(
    vyvoj_vyskytu(regex_2,popis_2),
    on="datum",
    how="full"
).with_columns(
    pl.when((pl.col('vyskytu') == 0) | (pl.col('vyskytu_right') == 0)).then(pl.lit(0)).otherwise(pl.lit(1)).alias('uplnost')
).with_columns(
    pl.when(pl.col('vyskytu') == 0).then(pl.lit(1)).otherwise(pl.col('vyskytu')).alias('vyskytu')
).with_columns(
    pl.when(pl.col('vyskytu_right') == 0).then(pl.lit(1)).otherwise(pl.col('vyskytu_right')).alias('vyskytu_right')
).group_by("datum").agg(
    pl.col(['vyskytu','vyskytu_right','uplnost']).sum()
).with_columns(
    (pl.col('vyskytu_right') / pl.col('vyskytu')).alias('vysledek')
).with_columns(
    pl.when(pl.col("vysledek") < 1).then(- (1 / pl.col("vysledek"))).otherwise(pl.col("vysledek")).alias("vysledek")
).with_columns(
        (pl.col("vysledek") - 1).alias("vysledek")  # Add this line
).with_columns(
    pl.when(pl.col("vysledek") < 0).then(pl.lit(popis_1)).when(pl.col("vysledek") == 0).then(pl.lit(None)).otherwise(pl.lit(popis_2)).alias("vitez")
).sort(by='datum')

In [9]:
def graf(frejm):
    return alt.Chart(
    frejm.to_pandas(),
    width=800,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N")
)

In [13]:
graf(vyvoj_vyskytu("(?i)kolegové a kolegyně","kolegové a kolegyně"))

alt.Chart(...)

In [185]:
pocet = 0

In [187]:
kolegove = vyvoj_vyskytu("(?i)kolegové.{1,12}kolegyně,","kolegové a kolegyně").join(
    vyvoj_vyskytu("(?i)kolegyně.{1,12}kolegové,","kolegyně a kolegové"),
    on="datum",
    how="full"
).filter(
    (pl.col("vyskytu") >= pocet) & (pl.col('vyskytu_right') >= pocet)
).with_columns(
    (pl.col('vyskytu_right') / pl.col('vyskytu')).alias('vysledek')
).with_columns(
    pl.when(pl.col("vysledek") < 1).then(- (1 / pl.col("vysledek"))).otherwise(pl.col("vysledek")).alias("vysledek")
)

kolegove

datum,vyskytu,pocet_slov,pomer,co,datum_right,vyskytu_right,pocet_slov_right,pomer_right,co_right,vysledek
date,u32,u32,f64,str,date,u32,u32,f64,str,f64
1918-01-01,0,172010,0.0,"""kolegové a kolegyně""",1918-01-01,0,172010,0.0,"""kolegyně a kolegové""",NaN
1919-01-01,0,1547459,0.0,"""kolegové a kolegyně""",1919-01-01,0,1547459,0.0,"""kolegyně a kolegové""",NaN
1920-01-01,0,1557497,0.0,"""kolegové a kolegyně""",1920-01-01,0,1557497,0.0,"""kolegyně a kolegové""",NaN
1921-01-01,1,1814119,5.5123e-7,"""kolegové a kolegyně""",1921-01-01,0,1814119,0.0,"""kolegyně a kolegové""",-inf
1922-01-01,0,1739976,0.0,"""kolegové a kolegyně""",1922-01-01,0,1739976,0.0,"""kolegyně a kolegové""",NaN
1923-01-01,0,1131296,0.0,"""kolegové a kolegyně""",1923-01-01,0,1131296,0.0,"""kolegyně a kolegové""",NaN
1924-01-01,1,1386775,7.2110e-7,"""kolegové a kolegyně""",1924-01-01,0,1386775,0.0,"""kolegyně a kolegové""",-inf
1925-01-01,0,989387,0.0,"""kolegové a kolegyně""",1925-01-01,0,989387,0.0,"""kolegyně a kolegové""",NaN
1926-01-01,0,1629693,0.0,"""kolegové a kolegyně""",1926-01-01,0,1629693,0.0,"""kolegyně a kolegové""",NaN


In [383]:
panove = vyvoj_vyskytu("(?i)pánové.{1,15}dámy,","pánové a dámy").join(
    vyvoj_vyskytu("(?i)dámy.{1,15}pánové,","dámy a pánové"),
    on="datum",
    how="full"
).filter(
    (pl.col("vyskytu") >= pocet) & (pl.col('vyskytu_right') > pocet)
).with_columns(
    (pl.col('vyskytu_right') / pl.col('vyskytu')).alias('vysledek')
).with_columns(
    pl.when(pl.col("vysledek") < 1).then(- (1 / pl.col("vysledek"))).otherwise(pl.col("vysledek")).alias("vysledek")
)

panove

datum,vyskytu,pocet_slov,pomer,co,datum_right,vyskytu_right,pocet_slov_right,pomer_right,co_right,vysledek
date,u32,u32,f64,str,date,u32,u32,f64,str,f64
1919-01-01,68,1547459,0.000044,"""pánové a dámy""",1919-01-01,19,1547459,0.000012,"""dámy a pánové""",-3.578947
1920-01-01,77,1557497,0.000049,"""pánové a dámy""",1920-01-01,55,1557497,0.000035,"""dámy a pánové""",-1.4
1921-01-01,66,1814119,0.000036,"""pánové a dámy""",1921-01-01,95,1814119,0.000052,"""dámy a pánové""",1.439394
1922-01-01,41,1739976,0.000024,"""pánové a dámy""",1922-01-01,96,1739976,0.000055,"""dámy a pánové""",2.341463
1923-01-01,20,1131296,0.000018,"""pánové a dámy""",1923-01-01,53,1131296,0.000047,"""dámy a pánové""",2.65
1924-01-01,26,1386775,0.000019,"""pánové a dámy""",1924-01-01,44,1386775,0.000032,"""dámy a pánové""",1.692308
1925-01-01,19,989387,0.000019,"""pánové a dámy""",1925-01-01,27,989387,0.000027,"""dámy a pánové""",1.421053
1926-01-01,40,1629693,0.000025,"""pánové a dámy""",1926-01-01,26,1629693,0.000016,"""dámy a pánové""",-1.538462
1927-01-01,18,1725378,0.00001,"""pánové a dámy""",1927-01-01,32,1725378,0.000019,"""dámy a pánové""",1.777778


In [379]:
poslanci = vyvoj_vyskytu("(?i)poslanci.{1,15}poslankyně,","poslanci a poslankyně").join(
    vyvoj_vyskytu("(?i)poslankyně.{1,15}poslanci,","poslankyně a poslanci"),
    on="datum",
    how="full"
).filter(
    (pl.col("vyskytu") >= pocet) & (pl.col('vyskytu_right') >= pocet)
).with_columns(
    (pl.col('vyskytu_right') / pl.col('vyskytu')).alias('vysledek')
).with_columns(
    pl.when(pl.col("vysledek") < 1).then(- (1 / pl.col("vysledek"))).otherwise(pl.col("vysledek")).alias("vysledek")
)

poslanci

datum,vyskytu,pocet_slov,pomer,co,datum_right,vyskytu_right,pocet_slov_right,pomer_right,co_right,vysledek
date,u32,u32,f64,str,date,u32,u32,f64,str,f64
1918-01-01,0,172010,0.0,"""poslanci a poslankyně""",1918-01-01,0,172010,0.0,"""poslankyně a poslanci""",NaN
1919-01-01,0,1547459,0.0,"""poslanci a poslankyně""",1919-01-01,0,1547459,0.0,"""poslankyně a poslanci""",NaN
1920-01-01,0,1557497,0.0,"""poslanci a poslankyně""",1920-01-01,0,1557497,0.0,"""poslankyně a poslanci""",NaN
1921-01-01,0,1814119,0.0,"""poslanci a poslankyně""",1921-01-01,0,1814119,0.0,"""poslankyně a poslanci""",NaN
1922-01-01,0,1739976,0.0,"""poslanci a poslankyně""",1922-01-01,0,1739976,0.0,"""poslankyně a poslanci""",NaN
1923-01-01,0,1131296,0.0,"""poslanci a poslankyně""",1923-01-01,0,1131296,0.0,"""poslankyně a poslanci""",NaN
1924-01-01,0,1386775,0.0,"""poslanci a poslankyně""",1924-01-01,0,1386775,0.0,"""poslankyně a poslanci""",NaN
1925-01-01,0,989387,0.0,"""poslanci a poslankyně""",1925-01-01,0,989387,0.0,"""poslankyně a poslanci""",NaN
1926-01-01,0,1629693,0.0,"""poslanci a poslankyně""",1926-01-01,0,1629693,0.0,"""poslankyně a poslanci""",NaN


In [381]:
soudruzi = vyvoj_vyskytu("(?i)soudruzi.{1,15}soudružky,","soudruzi a soudružky").join(
    vyvoj_vyskytu("(?i)soudružky.{1,15}soudruzi,","soudružky a soudruzi"),
    on="datum",
    how="full"
).filter(
    (pl.col("vyskytu") >= pocet) & (pl.col('vyskytu_right') >= pocet)
).with_columns(
    (pl.col('vyskytu_right') / pl.col('vyskytu')).alias('vysledek')
).with_columns(
    pl.when(pl.col("vysledek") < 1).then(- (1 / pl.col("vysledek"))).otherwise(pl.col("vysledek")).alias("vysledek")
)

soudruzi

datum,vyskytu,pocet_slov,pomer,co,datum_right,vyskytu_right,pocet_slov_right,pomer_right,co_right,vysledek
date,u32,u32,f64,str,date,u32,u32,f64,str,f64
1918-01-01,0,172010,0.0,"""soudruzi a soudružky""",1918-01-01,0,172010,0.0,"""soudružky a soudruzi""",NaN
1919-01-01,0,1547459,0.0,"""soudruzi a soudružky""",1919-01-01,0,1547459,0.0,"""soudružky a soudruzi""",NaN
1920-01-01,0,1557497,0.0,"""soudruzi a soudružky""",1920-01-01,0,1557497,0.0,"""soudružky a soudruzi""",NaN
1921-01-01,0,1814119,0.0,"""soudruzi a soudružky""",1921-01-01,0,1814119,0.0,"""soudružky a soudruzi""",NaN
1922-01-01,0,1739976,0.0,"""soudruzi a soudružky""",1922-01-01,0,1739976,0.0,"""soudružky a soudruzi""",NaN
1923-01-01,0,1131296,0.0,"""soudruzi a soudružky""",1923-01-01,0,1131296,0.0,"""soudružky a soudruzi""",NaN
1924-01-01,0,1386775,0.0,"""soudruzi a soudružky""",1924-01-01,0,1386775,0.0,"""soudružky a soudruzi""",NaN
1925-01-01,0,989387,0.0,"""soudruzi a soudružky""",1925-01-01,0,989387,0.0,"""soudružky a soudruzi""",NaN
1926-01-01,0,1629693,0.0,"""soudruzi a soudružky""",1926-01-01,0,1629693,0.0,"""soudružky a soudruzi""",NaN


In [385]:
len(", vážené paní")

13

In [537]:
do_grafu = pl.concat(
    [
        soudruzi, poslanci, panove
    ]
).with_columns(
    pl.when((pl.col('vyskytu') == 0) | (pl.col('vyskytu_right') == 0)).then(pl.lit(0)).otherwise(pl.lit(1)).alias('uplnost')
).with_columns(
    pl.when(pl.col('vyskytu') == 0).then(pl.lit(1)).otherwise(pl.col('vyskytu')).alias('vyskytu')
).with_columns(
    pl.when(pl.col('vyskytu_right') == 0).then(pl.lit(1)).otherwise(pl.col('vyskytu_right')).alias('vyskytu_right')
).group_by("datum").agg(
    pl.col(['vyskytu','vyskytu_right','uplnost']).sum()
).with_columns(
    (pl.col('vyskytu_right') / pl.col('vyskytu')).alias('vysledek')
).with_columns(
    pl.when(pl.col("vysledek") < 1).then(- (1 / pl.col("vysledek"))).otherwise(pl.col("vysledek")).alias("vysledek")
).with_columns(
    pl.when(pl.col("vysledek") < 0).then(pl.lit("m")).otherwise(pl.lit("f")).alias("vitez")
).sort(by='datum')

do_grafu

datum,vyskytu,vyskytu_right,uplnost,vysledek,vitez
date,u32,u32,i32,f64,str
1918-01-01,2,2,0,1.0,"""f"""
1919-01-01,70,21,1,-3.333333,"""m"""
1920-01-01,79,57,1,-1.385965,"""m"""
1921-01-01,68,97,1,1.426471,"""f"""
1922-01-01,43,98,1,2.27907,"""f"""
1923-01-01,22,55,1,2.5,"""f"""
1924-01-01,28,46,1,1.642857,"""f"""
1925-01-01,21,29,1,1.380952,"""f"""
1926-01-01,42,28,1,-1.5,"""m"""


In [389]:
len(df.filter(pl.col("text").str.contains("(?i)soudruzi a soudružky")))

266

In [313]:
len(df.filter(pl.col("text").str.contains("(?i)soudružky a soudruzi")))

6941

In [315]:
6941/266

26.093984962406015

In [619]:
g = alt.Chart(
    do_grafu.to_pandas(),
    width=330,
    height=330,
    title=alt.Title(
        "Pánové a dámy, nebo dámy a pánové?",
        subtitle=[
            "Ve 20. letech 20. století začínala v poslaneckých projevech kolektivní oslovení",
            "podobně často mužským tvarem jako ženským. Nejvýrazněji dostávaly ženy",
            "přednost za socialismu: oslovení „soudružky a soudruzi“ bylo 26× častější než",
            "„soudruzi a soudružky“.",
            "(Počítáme formy kolegové-kolegyně, soudruzi-soudružky a pánové-dámy.",
            "A to pouze ukončené čárkou, jež napovídá, byť nezaručuje, že jde o oslovení.)"
        ]
    )
).mark_bar(
    height=2.5,
).encode(
    alt.X(
        'vysledek:Q', 
        title="← kolikrát častěji jsou dříve osloveni muži | kolikrát častěji jsou dříve osloveny ženy →", 
        scale=alt.Scale(domain=[-350,350]), 
        axis=alt.Axis(
            labelExpr="replace(datum.label, '−', '') + '×'",
            titleAlign='right', 
            titleX=333, 
            orient="top")
    ),
    alt.Y('datum:T', title=None),
    alt.Color(
        "vitez:N", 
        scale=alt.Scale(
            range=['#67a587','#df635d']
        ), 
        legend=None
    ),
    alt.Opacity(
        'uplnost:Q',
        scale=alt.Scale(
            range=[1]
        ),
        legend=None
    )
)

g

alt.Chart(...)

In [621]:
g.save("grafy/panove_damy_graf.png", scale_factor=3)

In [149]:
alt.Chart(
    pl.concat(
        [kolegove, soudruzi, panove]
    ).to_pandas(),
    width=350,
    height=350,
    title=alt.Title(
        "Dámy a pánové, nebo pánové a dámy?",
        subtitle=["V Poslanecké sněmovně "]
    )
).mark_bar(
    height=2.5
).encode(
    alt.X(
        'vysledek:Q', 
        title="← kolikrát častěji jsou dříve osloveni muži | kolikrát častěji jsou dříve osloveny ženy →", 
        scale=alt.Scale(domain=[-450,450]), 
        axis=alt.Axis(
            labelExpr="datum.label + '×'", 
            titleAlign='right', 
            titleX=343, 
            orient="top")
    ),
    alt.Y('datum:T', title=None),
    alt.Color('co_right:N', 
              legend=alt.Legend(
                  title=None,
                  direction="horizontal",
                  orient="top",
                  columns=3),
              sort=['dámy a pánové','kolegyně a kolegové','soudružky a soudruzi']
             )
).resolve_scale(
    y='independent'
)

alt.Chart(...)

In [555]:
graf_vyvoje("(?i)automobil[^\w]","automobil","(?i)auto[^\w]","auto")

alt.Chart(...)

In [451]:
porovnani_vyskytu("(?i)pohlavek[^\w]","pohlavek","(?i)facka[^\w]","facka")

datum,vyskytu,vyskytu_right,uplnost,vysledek,vitez
date,u32,u32,i32,f64,str
1918-01-01,1,1,0,1.0,null
1919-01-01,1,1,0,1.0,null
1920-01-01,1,1,0,1.0,null
1921-01-01,1,1,0,1.0,null
1922-01-01,1,1,0,1.0,null
1923-01-01,1,1,0,1.0,null
1924-01-01,1,1,0,1.0,null
1925-01-01,1,1,0,1.0,null
1926-01-01,1,1,0,1.0,null


In [453]:
porovnani_vyskytu("(?i)továrník[^\w]","továrník","(?i)podnikatel[^\w]","podnikatel")

datum,vyskytu,vyskytu_right,uplnost,vysledek,vitez
date,u32,u32,i32,f64,str
1918-01-01,4,5,1,1.25,"""podnikatel"""
1919-01-01,18,39,1,2.166667,"""podnikatel"""
1920-01-01,8,21,1,2.625,"""podnikatel"""
1921-01-01,9,61,1,6.777778,"""podnikatel"""
1922-01-01,12,38,1,3.166667,"""podnikatel"""
1923-01-01,3,44,1,14.666667,"""podnikatel"""
1924-01-01,6,52,1,8.666667,"""podnikatel"""
1925-01-01,3,21,1,7.0,"""podnikatel"""
1926-01-01,7,30,1,4.285714,"""podnikatel"""


In [457]:
porovnani_vyskytu("(?i)špitál[^\w]","špitál","(?i)nemocnice[^\w]","nemocnice")

datum,vyskytu,vyskytu_right,uplnost,vysledek,vitez
date,u32,u32,i32,f64,str
1918-01-01,1,1,0,1.0,null
1919-01-01,1,49,0,49.0,"""nemocnice"""
1920-01-01,1,77,1,77.0,"""nemocnice"""
1921-01-01,1,83,0,83.0,"""nemocnice"""
1922-01-01,1,51,0,51.0,"""nemocnice"""
1923-01-01,1,54,0,54.0,"""nemocnice"""
1924-01-01,1,35,0,35.0,"""nemocnice"""
1925-01-01,1,155,0,155.0,"""nemocnice"""
1926-01-01,1,13,0,13.0,"""nemocnice"""


In [467]:
porovnani_vyskytu("(?i)miliardář\w{0,3}","miliardář","(?i)oligarch[^i]\w{0,3}","oligarcha")

datum,vyskytu,vyskytu_right,uplnost,vysledek,vitez
date,u32,u32,i32,f64,str
1918-01-01,1,1,0,1.0,null
1919-01-01,2,1,0,-2.0,"""miliardář"""
1920-01-01,3,1,0,-3.0,"""miliardář"""
1921-01-01,10,1,0,-10.0,"""miliardář"""
1922-01-01,2,1,1,-2.0,"""miliardář"""
1923-01-01,1,1,0,1.0,null
1924-01-01,2,3,1,1.5,"""oligarcha"""
1925-01-01,4,1,0,-4.0,"""miliardář"""
1926-01-01,1,1,0,1.0,null


In [589]:
def graf_vyvoje(regex_1,popis_1,regex_2,popis_2):
    return alt.Chart(
        porovnani_vyskytu(
            regex_1,popis_1,regex_2,popis_2
        ).to_pandas(), 
        width=350,
    height=350
    ).mark_bar(
    height=2.5
).encode(
    alt.X(
        'vysledek:Q', 
        title=f"← častěji {popis_1} | častěji {popis_2} →", 
        axis=alt.Axis(
            labelExpr="datum.label + '×'", 
            titleAlign='right', 
            orient="top")
    ),
    alt.Y('datum:T', title=None),
    alt.Color(
            'vitez:N',
        scale=alt.Scale(
            range=['#67a587','#df635d']
        ),
            legend=None  # Hide the color legend since it's self-explanatory
        )
).resolve_scale(
    y='independent'
)

In [607]:
graf_vyvoje("(?i)vzducholoď[^\w]","vzducholoď","(?i)letadlo","letadlo")

alt.Chart(...)

In [609]:
graf_vyvoje("(?i)míč[^\w]","míč","(?i)bal[oó]n[^\w]","balon")

alt.Chart(...)

In [611]:
graf_vyvoje("(?i)uheln[éíý]","uhelné","(?i)solární","solární")

alt.Chart(...)

In [613]:
graf_vyvoje("(?i)uheln[áéíýo]","uhelné","(?i)atomov[áéíoý]","atomové")

alt.Chart(...)

In [489]:
graf_vyvoje("(?i)zbabělec[^\w]","zbabělec","(?i)srab[^\w]","srab")

alt.Chart(...)

In [493]:
graf_vyvoje("(?i)chřipka[^\w]","chřipka","(?i)covid[^\w]","covid")

alt.Chart(...)

In [505]:
graf_vyvoje("Moskva[^\w]","Moskva","Brusel[^\w]","Brusel")

alt.Chart(...)

In [507]:
graf_vyvoje("(?i)dívka[^\w]","dívka","(?i)holka[^\w]","holka")

alt.Chart(...)

In [509]:
graf_vyvoje("(?i)mládenec[^\w]","mládenec","(?i)mladý muž[^\w]","mladý muž")

alt.Chart(...)

In [511]:
graf_vyvoje("(?i)telegraf[^\w]","telegraf","(?i)internet[^\w]","internet")

alt.Chart(...)

In [513]:
graf_vyvoje("(?i)motocykl[^\w]","motocykl","(?i)motorka[^\w]","motorka")

alt.Chart(...)

In [515]:
graf_vyvoje("(?i)homosexu","homosexuál","(?i)gay\w{0,3}","gay")

alt.Chart(...)